In [61]:
from colorama import Fore, Back, Style
import re
from  dataclasses import dataclass


In [62]:

@dataclass
class MemoryInfo():
    """MicroPython Memory information map"""
    total: int = 0
    """Total memory"""
    used: int = 0
    """Used memory"""
    free: int = 0
    """Free memory"""
    one_blocks: int = 0
    """Number of 1-blocks"""
    two_blocks: int  = 0
    """Number of 2-blocks"""
    max_block_size: int = 0
    """Largest available block"""	
    max_free_size: int = 0
    """largest free block"""
    memory_map: str = ""
    """Memory map showing allocated and free areas"""
    lines_free: int = 0
    show_free: bool = True
    columns:int  = 2
    rainbow: bool = False
 
    def __init__(self, mem_info, show_free=False, columns = 4):
        # sourcery skip: use-named-expression
        """Parse the memory map"""

        if issubclass(type(mem_info), list):
            mem_info = "\n".join(mem_info)
        elif issubclass(type(mem_info), str):
            mem_info = str(mem_info)

        self.show_free = show_free
        self.columns = columns
        self._color_num = 0

        match_1 = re.search(r'GC: total: (\d+), used: (\d+), free: (\d+)', mem_info)
        if not match_1:
            raise ValueError('Not recognized as a valid Micropython memory info')
        self.total, self.used, self.free = [int(x) for x in match_1.groups()]
        # find the used blocks
        match_2 = re.search(r' No. of 1-blocks: (\d+), 2-blocks: (\d+), max blk sz: (\d+), max free sz: (\d+)', mem_info)
        if match_2:
            self.one_blocks, self.two_blocks, self.max_block_size, self.max_free_size = [int(x) for x in match_2.groups()]
        match_3 = re.search(r'\((.*) lines all free\)', mem_info)
        if match_3:
            self.lines_free = int(match_3.groups(0)[0])

        self._raw_map = re.findall(r'^[0-9a-fA-F]*\: (.*)', mem_info, flags=re.MULTILINE) 
        self.memory_map = "".join(self._raw_map)

        self.update_map()

    def update_map(self):
        "show/hide the free space in the memory map" 
        _map = self._raw_map.copy()
        # TODO: Where to insert the free lines - currently just use -1 ...
        if self.show_free :
            for _ in range(self.lines_free):
                _map.insert(-1, '.'*64)
        self.memory_map = "".join(_map)

    def _repr_pretty_(self, pp, cycle):
        "print a colored version of the memory map"


        text = f"{Fore.WHITE}{Back.BLACK}Memory Used: 0x{self.used:X} of Total: 0x{self.total:X}\nFree: 0x{self.free:X} {self.free/self.total:.1%}\n"
        COL_WIDTH = 64
        width = COL_WIDTH * self.columns
        # self.update_map()

        # BG_COLORS = [Back.BLUE, Back.RED, Back.YELLOW, Back.MAGENTA, Back.CYAN]
        # self.back_index = 0
        #  # Back.GREEN, Back.WHITE
        # def new_BG (self, c=''):
        #     self.back_index = (self.back_index + 1) % len(BG_COLORS)
        #     return BG_COLORS[self.back_index]


        color = Fore.WHITE
        for i in range(len(self.memory_map)): 
            # '=' keeps the same color
            if self.memory_map[i] != '=': 
                color = self.color(self.memory_map[i])
            text += color + self.memory_map[i]
            # columns
            if (i+1)%COL_WIDTH == 0:
                text += f'{Style.RESET_ALL} '
            # rows
            if (i+1)%width == 0:
                text += Style.RESET_ALL + '\n'
        # now pretty print the memory map
        # pp.text(text+Style.RESET_ALL)
        pp.text(text)



    def color(self, c):
        # ====== =================
        # Symbol Meaning
        # ====== =================
        #    .   free block
        #    h   head block
        #    =   tail block
        #    m   marked head block
        #    T   tuple
        #    L   list
        #    D   dict
        #    F   float
        #    B   byte code
        #    M   module
        #    S   string or bytes
        #    A   bytearray
        # ====== =================
        BG_COLORS = [Back.BLUE, Back.RED, Back.MAGENTA, Back.CYAN]
        fg = Fore.BLACK
        bg =  Back.RED
        if c == '.':
            fg = Fore.GREEN
            bg = Back.GREEN
        elif c.isupper():
            fg = Fore.WHITE
        else:
            fg =  Fore.BLACK
        if c in 'TSLDFABh':
            if self.rainbow:
                bg = BG_COLORS[self._color_num]
                self._color_num = (self._color_num + 1) % len(BG_COLORS)
            else:
                bg = Back.RED
        elif c == 'M':
            fg = Fore.BLACK
            bg = Back.CYAN
        return fg + bg 
        

mem_lst = %mpy import micropython; micropython.mem_info(True)
mem=MemoryInfo(mem_lst.data )
mem

Memory Used: 0x18A0 of Total: 0x2EE40
Free: 0x2D5A0 96.7%
h=Mhhh=DhhhhhhDBDBhhh===BhB=Dh====B=BBBBBBhB=BhB=BBBBB=hBhB=Bh== =DB=h===========h===================BBBhhBh=h=====h============= ====h=========================================================== ====h=========================================================== 
====hhh=h=h=hhBh=..h=...h=h=.......h=........................... ........................................h....................... ...................h=======h========............................ ...........................................................h==== 
h=============================================================== ========........................................................ ......................................................hh===..... .h.............................................................. 
...........h=======....hh..B.............h...................... ....................................

In [63]:
# consume some more memory

%mpy print(help('modules'))
# %mpy import gc; gc.collect()

mem_lst = %mpy import micropython; micropython.mem_info(True)
mem_2=MemoryInfo(mem_lst.data )
mem_2



Memory Used: 0x1D80 of Total: 0x2EE40
Free: 0x2D0C0 96.1%
h=Mhhh=DhhhhhhDBDBhhh===BhB=Dh====B=BBBBBBhB=BhB=BBBBB=hBhB=Bh== =DB=h===========h===================BBBhhBh=h=====h============= ====h=========================================================== ====h=========================================================== 
====hhh=h=h=hhBh=hhh=hhhh=h=BLShShSh=hShSh=Sh==h================ ===============hShShShShShShShhhhB..h...h.....h=..h=............ ...................h=======h========............................ ...........................................................h==== 
h=============================================================== ========........................................................ ......................................................hh===..... .h.............................................................. 
...........h=======....hh..B.............h...................... ....................................

In [71]:
%%micropython
import gc
import micropython

gc.disable()

foo = 2
for i in range(1500):
    foo = foo + foo


[]

In [72]:

mem_lst = %mpy micropython.mem_info(True)
mem_1=MemoryInfo(mem_lst.data, show_free=True )

mem_1


Memory Used: 0x2E180 of Total: 0x2EE40
Free: 0xCC0 1.7%
h=Mhhh=DhhhhhhDBDBhhh===BhB=Dh====B=BBBBBBhB=BhB=BBBBB=hBhB=Bh== =DB=h===========h===================BBBhhBh=h=====h============= ====h=========================================================== ====h=========================================================== 
====hhh=h=h=hhhh=Bhh=hhhh=h=BhhhhBhhhhhhh=Shh===Bhhhhh=h=BShhhBh hhhh=h======hhhhhhhh===hh===hhhhh======hhhh===hhhhhhhhhhhhhhhhhh hhhhhhhhhhhhhhhhhhhh=======h========h=========================== ============================================hhhhhhhhhhhhhhhh==== 
h=============================================================== ========h======================================================= ================hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh=hh=hh=hh=hh= 
hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=h h=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=hh =hh=hh=hh=hh=hh=hh=hh=hh=hh=hh=

In [66]:
mem_2

Memory Used: 0x1730 of Total: 0x2EE40
Free: 0x2D710 96.9%
h=Mhhh=DhhhhhhDBDBh.h===B.B=Dh====B=BBBBBB.B=B.B=BBB.B=.B.B=Bh== =DB=h===========h===================BBB...h=h=====h============= ====h=========================================================== ====h=========================================================== 
====..h=..h=.................................................... .........................................h...................... ...................h=======h========h=========================== ============================================...............h==== 
......................................................hh===..... ..............hh===========....hh..B.............h.............. ................................................................ ....................................

In [67]:
mem_lst = %mpy import micropython; micropython.mem_info(False)
print(type(mem_lst))
mem_lst


<class 'micropython_magic.octarine.PrettyOutput'>


['stack: 500 out of 7936', 'GC: total: 192064, used: 6080, free: 185984', ' No. of 1-blocks: 45, 2-blocks: 14, max blk sz: 72, max free sz: 10131', 'GC memory layout; from 20009520:', '00000000: h=Mhhh=DhhhhhhDBDBhhh===BhB=Dh====B=BBBBBBhB=BhB=BBBBB=.B.B=Bh==', '00000400: =DB=h===========h===================BBB...h=h=====h=============', '00000800: ====h===========================================================', '00000c00: ====h===========================================================', '00001000: ====..h=h=h=.......h=...........................................', '00001400: .........................................h......................', '00001800: ...................h=======h========h===========================', '00001c00: ============================================...............h====', '       (18 lines all free)', '00006800: ......................................................hh===.....', '       (158 lines all free)', '0002e400: ..............hh===========....hh..B......

In [68]:
mem_lst = %mpy import micropython; micropython.mem_info()
mem_lst

['stack: 500 out of 7936', 'GC: total: 192064, used: 6224, free: 185840', ' No. of 1-blocks: 50, 2-blocks: 16, max blk sz: 72, max free sz: 10131']

In [69]:
mem_info=MemoryInfo(mem_lst)
mem_info

TypeError: expected string or bytes-like object, got 'PrettyOutput'

In [ ]:
op = !dir
type(dir)


In [ ]:
import colorama
from colorama import Fore, Back, Style
# prrint a colored version of the memory map where the used memory is colored red and the free memory is colored green. 
# free memory is a . and used memory is a h


# ====== =================
# Symbol Meaning
# ====== =================
#    .   free block
#    h   head block
#    =   tail block
#    m   marked head block
#    T   tuple
#    L   list
#    D   dict
#    F   float
#    B   byte code
#    M   module
#    S   string or bytes
#    A   bytearray
# ====== =================


width = 256
self._col = 64

color = Fore.WHITE
for i in range(len(update_map)): 
    if update_map[i] == '.': 
        color = Fore.GREEN + Back.GREEN
    elif update_map[i] == 'h': # head block
        color = Fore.BLACK+ Back.RED
    elif update_map[i] == 'B': # byte code
        color = Fore.WHITE + Back.BLUE
    elif update_map[i] in 'TSLDFA': # tuple
        color = Fore.WHITE + Back.BLUE
    elif update_map[i] == 'M': # module
        color = Fore.WHITE + Back.RED
    print(color + update_map[i], end='')

    # columns
    if (i+1)%self._col == 0:
        print(f'{Style.RESET_ALL} ', end='')
    # rows
    if (i+1)%width == 0:
        print()

print(Style.RESET_ALL)


